<a href="https://colab.research.google.com/github/SamuelA-R/Gasolina-SerieTemporal/blob/main/GASOLINA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('gasolina_2000+.csv')
df2 = pd.read_csv('gasolina_2010+.csv')

<ipython-input-2-c57a957ad8cf>:2: DtypeWarning: Columns (12,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('gasolina_2010+.csv')


In [3]:
def tamanho(x):
    return print(f'Os dados tem {x.shape[0]} linhas e {x.shape[1]} colunas')

In [4]:
tamanho(df1)
tamanho(df2)

Os dados tem 42638 linhas e 19 colunas
Os dados tem 78185 linhas e 19 colunas


In [5]:
df = pd.concat([df1, df2], axis=0)
tamanho(df)

Os dados tem 120823 linhas e 19 colunas


Agora vamos construir uma tabela para saber a variação percentual do preço da gasolina ano a ano para o estado do RJ

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120823 entries, 0 to 78184
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     120823 non-null  int64  
 1   DATA INICIAL                   120823 non-null  object 
 2   DATA FINAL                     120823 non-null  object 
 3   REGIÃO                         120823 non-null  object 
 4   ESTADO                         120823 non-null  object 
 5   PRODUTO                        120823 non-null  object 
 6   NÚMERO DE POSTOS PESQUISADOS   120823 non-null  int64  
 7   UNIDADE DE MEDIDA              120823 non-null  object 
 8   PREÇO MÉDIO REVENDA            120823 non-null  float64
 9   DESVIO PADRÃO REVENDA          120823 non-null  float64
 10  PREÇO MÍNIMO REVENDA           120823 non-null  float64
 11  PREÇO MÁXIMO REVENDA           120823 non-null  float64
 12  MARGEM MÉDIA REVENDA           

In [7]:
#criando coluna mês e ano-mes
df['DATA FINAL'] = pd.to_datetime(df['DATA FINAL'])
df['ANO-MES'] = df['DATA FINAL'].apply(lambda x: '{}'.format(x.year)) + df['DATA FINAL'].apply(lambda x: '/{:02d}'.format(x.month))
df['MES'] = df['DATA FINAL'].apply(lambda x: x.month)
df_rio = df[df['ESTADO'] == 'RIO DE JANEIRO']

In [8]:
#Agrupar por ano e mês, mostrando apenas o último mês
df_month_rio = df_rio.groupby('ANO-MES')[['PREÇO MÉDIO REVENDA', 'MES']].last()

In [9]:
#Variação percentual de cada ano (Nessa tabela podemos observar como o preço subiu)
(df_month_rio[df_month_rio['MES'] == 12] / df_month_rio[df_month_rio['MES'] == 12].shift(1) - 1) * 100

,PREÇO MÉDIO REVENDA,MES
ANO-MES,,
2004/12,NaN,NaN
2005/12,12.338063,0.0
2006/12,-0.878638,0.0
2007/12,0.720222,0.0
2008/12,14.466447,0.0
2009/12,-4.949543,0.0
2010/12,0.202224,0.0
2011/12,1.261352,0.0
2012/12,5.630294,0.0


Criando tabela de serie temporal com a diferença absoluta e percentual entre os preços médio de revenda mais baratos e mais caros, onde mostra o estado e a data.

In [10]:
df_max = df.groupby('ANO-MES').max()['PREÇO MÉDIO REVENDA']
df_min = df.groupby('ANO-MES').min()['PREÇO MÉDIO REVENDA']

idx_max = df.groupby('ANO-MES')['PREÇO MÉDIO REVENDA'].idxmax()
idx_min = df.groupby('ANO-MES')['PREÇO MÉDIO REVENDA'].idxmin()

<ipython-input-10-1cb9b8e57b24>:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  df_max = df.groupby('ANO-MES').max()['PREÇO MÉDIO REVENDA']
<ipython-input-10-1cb9b8e57b24>:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  df_min = df.groupby('ANO-MES').min()['PREÇO MÉDIO REVENDA']


In [11]:
#Criando DataFrame vazio para anexar os dados
df_diff = pd.DataFrame()

In [12]:
#Diferença absoluta entre os valores máximos e minimos por ano e mês
df_diff['abs_diff'] = df_max - df_min
df_diff['abs_diff(%)'] = (df_max - df_min) / df_min * 100

df_diff['max'] = df_max
df_diff['min'] = df_min


In [13]:
#Adicionando os estados
df.loc[idx_max, :][['ESTADO', 'PREÇO MÉDIO REVENDA']]

,ESTADO,PREÇO MÉDIO REVENDA
18529,ACRE,39.328
18529,ACRE,3.259
18610,ACRE,39.046
18610,MARANHAO,2.300
18826,ACRE,39.081
...,...,...
75783,MATO GROSSO,98.220
76305,AMAPA,101.444
77018,ACRE,107.500
77562,MATO GROSSO,105.194


In [14]:
df_diff

,abs_diff,abs_diff(%),max,min
ANO-MES,,,,
2004/05,38.562,5034.203655,39.328,0.766
2004/06,38.159,4302.029312,39.046,0.887
2004/07,38.182,4247.163515,39.081,0.899
2004/08,38.053,3809.109109,39.052,0.999
2004/09,38.031,3806.906907,39.030,0.999
...,...,...,...,...
2021/01,95.521,3539.125602,98.220,2.699
2021/02,98.645,3524.294391,101.444,2.799
2021/03,104.751,3810.512914,107.500,2.749
